### Data Cleaning for Ames Housing Dataset

Apply cleaning pipeline and validate results.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add the project root to Python path
# This allows importing from src.data.cleaning
# Works from any directory - finds housing_price_project/ automatically
def find_project_root():
    """Find the housing_price_project directory."""
    current = Path.cwd().resolve()
    
    # Check if we're in notebooks/ - go up one level
    if current.name == 'notebooks':
        parent = current.parent
        if (parent / 'src').exists() and (parent / 'notebooks').exists():
            return parent
    
    # Search up the directory tree for housing_price_project
    path = current
    while path != path.parent:
        if path.name == 'housing_price_project' and (path / 'src').exists():
            return path
        path = path.parent
    
    # If not found, assume current directory is project root
    return current

project_root = find_project_root()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
from src.data.cleaning import clean_ames_data, create_ames_cleaning_config, print_cleaning_report, SimpleImputer, DomainOutlierFilter

In [4]:
DATA_PATH = Path("../data/raw/ames.csv")
df_raw = pd.read_csv(DATA_PATH)

print(f"Raw data: {df_raw.shape}")
print(f"Raw data: {df_raw.head()}")
print(f"Missing values: {df_raw.isnull().sum()}")



Raw data: (2930, 82)
Raw data:    Order        PID  area   price  MS.SubClass MS.Zoning  Lot.Frontage  \
0      1  526301100  1656  215000           20        RL         141.0   
1      2  526350040   896  105000           20        RH          80.0   
2      3  526351010  1329  172000           20        RL          81.0   
3      4  526353030  2110  244000           20        RL          93.0   
4      5  527105010  1629  189900           60        RL          74.0   

   Lot.Area Street Alley  ... Screen.Porch Pool.Area Pool.QC  Fence  \
0     31770   Pave   NaN  ...            0         0     NaN    NaN   
1     11622   Pave   NaN  ...          120         0     NaN  MnPrv   
2     14267   Pave   NaN  ...            0         0     NaN    NaN   
3     11160   Pave   NaN  ...            0         0     NaN    NaN   
4     13830   Pave   NaN  ...            0         0     NaN  MnPrv   

  Misc.Feature Misc.Val Mo.Sold Yr.Sold Sale.Type Sale.Condition  
0          NaN        0       

In [5]:
df_clean, report = clean_ames_data(df_raw)

print_cleaning_report(report)


DATA CLEANING REPORT

Original: 2930 rows × 82 columns
Final:    2685 rows × 80 columns
Rows removed: 245
Remaining nulls: 0

Steps performed:
  • Dropped columns: ['Order', 'PID']
  • Domain filter removed 245 rows
  • Filled NA with 'None' for 14 columns
  • Filled NA with 0 for 10 columns
  • Imputed Lot.Frontage with median=67.00
  • Imputed Mas.Vnr.Type with mode='BrkFace'
  • Imputed Electrical with mode='SBrkr'
  • Imputed MS.Zoning with mode='RL'

Domain filter details:
    original_rows: 2930
    Remove partial sales: 245
    Remove zero/null prices: 0
    final_rows: 2685
    total_removed: 245


In [6]:
if 'Sale.Condition' in df_clean.columns:
    partial_count = (df_clean['Sale.Condition'] == 'Partial').sum()
    print(f"\n✓ Partial sales remaining: {partial_count}")
    assert partial_count == 0, "Partial sales should be removed!"

# Check 2: Missing values handled
remaining_missing = df_clean.isnull().sum()
cols_with_missing = remaining_missing[remaining_missing > 0]

if len(cols_with_missing) == 0:
    print("✓ No missing values remain!")
else:
    print(f"\n⚠ Columns still with missing values:")
    print(cols_with_missing)

# Check 3: Target variable is valid
print(f"\n✓ Price range: ${df_clean['price'].min():,.0f} - ${df_clean['price'].max():,.0f}")
print(f"✓ Price median: ${df_clean['price'].median():,.0f}")

# Check 4: Data types look correct
print(f"\n✓ Numeric columns: {len(df_clean.select_dtypes(include=[np.number]).columns)}")
print(f"✓ Categorical columns: {len(df_clean.select_dtypes(include=['object']).columns)}")




✓ Partial sales remaining: 0
✓ No missing values remain!

✓ Price range: $12,789 - $755,000
✓ Price median: $155,000

✓ Numeric columns: 37
✓ Categorical columns: 43


In [7]:
OUTPUT_PATH = Path("../data/processed/ames_cleaned.csv")
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

df_clean.to_csv(OUTPUT_PATH, index=False)
print(f"\n✓ Cleaned data saved to: {OUTPUT_PATH}")


✓ Cleaned data saved to: ../data/processed/ames_cleaned.csv
